In [1]:
# fix numpy in colab
import numpy
from IPython.display import clear_output

# fix triton in colab
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/etc/alternatives/cuda/targets/x86_64-linux/include:/usr/include/python3.6m:$LD_LIBRARY_PATH"
!export LIBRARY_PATH="/etc/alternatives/cuda/lib64/stubs"
# !ldconfig /etc/alternatives/cuda/lib64/lib64-nvidia

# !git clone https://github.com/dvmazur/mixtral-offloading.git --quiet
# !cd mixtral-offloading && pip install -q -r requirements.txt
!huggingface-cli download lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo --quiet --local-dir Mixtral-8x7B-Instruct-v0.1-offloading-demo

clear_output()

In [11]:

import os, sys
script_dir = os.getcwd()
module_path = script_dir
for _ in range(1):
    module_path = os.path.abspath(os.path.join(module_path, '../'))
    if module_path not in sys.path:
        sys.path.insert(0,module_path)
        
sys.path.append("mixtral-offloading")
import torch
from torch.nn import functional as F
from hqq.core.quantize import BaseQuantizeConfig
from huggingface_hub import snapshot_download
from IPython.display import clear_output
from tqdm.auto import trange
from transformers import AutoConfig, AutoTokenizer
from transformers.utils import logging as hf_logging
import time
from src.build_model import OffloadConfig, QuantConfig, build_model

In [3]:
# This will reload the imported modules (e.g. get_decode_model_characterstics) every time you execute the jupyter cells, so that you don't need to restart the notebook after updating the source codes.
%load_ext autoreload
%autoreload 2  

In [6]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
quantized_model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"
state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-demo"

config = AutoConfig.from_pretrained(quantized_model_name)

device = torch.device("cuda:0")

##### Change this to 5 if you have only 12 GB of GPU VRAM #####
offload_per_layer = 4
# offload_per_layer = 5
###############################################################

num_experts = config.num_local_experts

offload_config = OffloadConfig(
    main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
    offload_size=config.num_hidden_layers * offload_per_layer,
    buffer_size=4,
    offload_per_layer=offload_per_layer,
)


attn_config = BaseQuantizeConfig(
    nbits=4,
    group_size=64,
    quant_zero=True,
    quant_scale=True,
)
attn_config["scale_quant_params"]["group_size"] = 256


ffn_config = BaseQuantizeConfig(
    nbits=2,
    group_size=16,
    quant_zero=True,
    quant_scale=True,
)
quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)


model = build_model(
    device=device,
    quant_config=quant_config,
    offload_config=offload_config,
    state_path=state_path,
    routing_strategy="THRESHOLDING",
    routing_threshold=0.05
)

Loading experts: 100%|██████████| 32/32 [00:10<00:00,  3.02it/s]


In [5]:
model

NameError: name 'model' is not defined

In [15]:
from transformers import TextStreamer


tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
past_key_values = None
sequence = None

seq_len = 0
# while True:
print("User: ", end="")
user_input = "Where is Georgia Tech? What is the name of its mascot?"
print("\n")

user_entry = dict(role="user", content=user_input)
input_ids = tokenizer.apply_chat_template([user_entry], return_tensors="pt").to(device)

if past_key_values is None:
  attention_mask = torch.ones_like(input_ids)
else:
  seq_len = input_ids.size(1) + past_key_values[0][0][0].size(1)
  attention_mask = torch.ones([1, seq_len - 1], dtype=torch.int, device=device)

print("Mixtral: \n", end="")


# sequence = result["sequences"]
# past_key_values = result["past_key_values"]

User: 

Mixtral: 


## DEFAULT with 4 Experts on GPU

In [18]:
start_time = time.time()
result = model.generate(
  input_ids=input_ids,
  attention_mask=attention_mask,
  past_key_values=past_key_values,
  streamer=streamer,
  do_sample=True,
  temperature=0.9,
  top_p=0.9,
  min_new_tokens=200,
  max_new_tokens=200,
  pad_token_id=tokenizer.eos_token_id,
  return_dict_in_generate=True,
  output_hidden_states=True,
)
latency = time.time() - start_time

print(f"Total Latency :{latency} sec ")

Georgia Tech, officially known as the Georgia Institute of Technology, is located in Atlanta, Georgia, in the United States. The institution is part of the University System of Georgia and is renowned for its strong programs in engineering, computing, and the sciences.

The Georgia Tech Yellow Jackets' mascot is a yellow jacket, specifically a modified version of a hornet. The mascot is often depicted in a stylized, cartoon form with a black and gold color scheme, reflecting the school's colors. The nickname for the mascot is "Buzz," and the school's official mascot is known as the "Georgia Tech Yellow Jacket."

The school's mascot is a source of pride for the student body, and the yellow jacket is a symbol that is used extensively in school spirit items, clothing, and merchandise. Georgia Tech's mascot is a beloved representation of the
Total Latency :92.60623288154602 sec 


## Thresholding with 4 Experts on GPU

In [17]:
start_time = time.time()
result = model.generate(
  input_ids=input_ids,
  attention_mask=attention_mask,
  past_key_values=past_key_values,
  streamer=streamer,
  do_sample=True,
  temperature=0.9,
  top_p=0.9,
  min_new_tokens=200,
  max_new_tokens=200,
  pad_token_id=tokenizer.eos_token_id,
  return_dict_in_generate=True,
  output_hidden_states=True,
)
latency = time.time() - start_time

print(f"Total Latency :{latency} sec ")

Georgia Tech, officially known as the Georgia Institute of Technology, is a public research university located in Atlanta, Georgia, in the United States. The university is part of the University System of Georgia and is a top-ranking institution in the fields of engineering, computing, and related sciences.

The mascot of Georgia Tech is a yellow-colored, English-speaking bird known as the Georgia Tech Yellow Jacket. The mascot is depicted as a fierce, energetic, and determined creature, which is intended to reflect the spirit of the university's students, faculty, and alumni. The Yellow Jacket mascot has been a fixture of Georgia Tech athletics and campus life for many decades, and it continues to be a source of pride and identity for the institution and its community.

The Georgia Tech Yellow Jackets are the university's intercollegiate athletic teams, which compete in the National Collegiate Athletic Association (NC
Total Latency :81.12173986434937 sec 


## Calculate Perplexity

In [ ]:
from evaluate.loading import evaluation_module_factory
from datasets import DownloadConfig, DownloadMode, Version
!pip install -U evaluate
from utils.perplexity_local_model import Perplexity
from typing import Optional, Union, List


def calculate_perplexity(predictions: List[str], model, tokenizer):
    config_name: Optional[str] = None
    module_type: Optional[str] = None
    process_id: int = 0
    num_process: int = 1
    cache_dir: Optional[str] = None
    experiment_id: Optional[str] = None
    keep_in_memory: bool = False
    download_config: Optional[DownloadConfig] = None
    download_mode: Optional[DownloadMode] = None
    revision: Optional[Union[str, Version]] = None
    
    perplexity_module = evaluation_module_factory(
        "perplexity", module_type=module_type, revision=revision, download_config=download_config, download_mode=download_mode
    )
    
    perplexity = Perplexity(
        config_name=config_name,
        process_id=process_id,
        num_process=num_process,
        cache_dir=cache_dir,
        keep_in_memory=keep_in_memory,
        experiment_id=experiment_id,
        hash=perplexity_module.hash
    )
    
    if module_type and module_type != perplexity.module_type:
        raise TypeError(
            f"No module of module type '{module_type}' not found for 'perplexity' locally, or on the Hugging Face Hub. Found module of module type '{perplexity.module_type}' instead."
        )
    
    # Download and prepare resources for the metric
    perplexity.download_and_prepare(download_config=download_config)
    
    # predictions, model, tokenizer, batch_size: int = 16, add_start_token: bool = True, device=None, max_length=None
    
    return perplexity.compute(predictions=predictions, add_start_token=False, model=model, tokenizer=tokenizer)['mean_perplexity']

calculate_perplexity(["Hello world is a common programming print statement.", "Perplexity is only useful within the same model"], model, tokenizer)

In [ ]:
# With help from GPT
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling

# Load the C4 dataset
dataset = load_dataset("c4", "en", split="validation")

In [ ]:
## Evaluation function
def evaluate_model_on_dataset(model, tokenizer, dataset):
    macro_batch_size = 512
    running_avg_pplx = None
    n = None
    
    for batch in dataset.iter(batch_size=macro_batch_size):
        texts = batch['text']
        next_batch_avg_pplx = calculate_perplexity(texts, model, tokenizer)
        if running_avg_pplx is not None:
            running_avg_pplx = (running_avg_pplx * n + next_batch_avg_pplx * len(texts)) / (n + len(texts))
            n += len(texts)
        else:
            running_avg_pplx = next_batch_avg_pplx
            n = len(texts)
    
    return running_avg_pplx, n


## Get perplexity on C4 using non-instruction fine-tuned model
routing_strategies = ['DEFAULT', 'THRESHOLDING', 'BIASING']

for routing_strategy in routing_strategies:
    dataset = load_dataset("c4", "en", split="validation")
    model_name = "mistralai/Mixtral-8x7B-v0.1"
    model = build_model(
        device=device,
        quant_config=quant_config,
        offload_config=offload_config,
        state_path=state_path,
        routing_strategy=routing_strategy,
        routing_threshold=0.05,
        model_name=model_name
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    mean_pplx, n = evaluate_model_on_dataset(model, tokenizer, dataset)
    print(f"{routing_strategy} on C4 dataset | Avg perplexity: {mean_pplx}, n_samples = {n}")

### Using residency information

## Get accuracy on MMLU using instruction fine-tuned model

### Using default routing

### Using thresholding

### Using residency information